In [1]:
import re
import cx_Oracle
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.dashboard_objs as dashboard

In [2]:
user_name = 'reviwe_ihor'
password = 'password'
server = 'xe'

In [ ]:
def fileId_from_url(url):
    """Return fileId from a url."""
    raw_fileId = re.findall("~[A-z.]+/[0-9]+", url)[0][1: ]
    return raw_fileId.replace('/', ':')

In [ ]:
connection = cx_Oracle.connect(user_name, password, server)

### Вивести довжину описа кожної програми

In [ ]:
sql_1 = '''
SELECT
    soft_name,
    version,
    length(description) len_desc
FROM
    software
'''

In [ ]:
df = pd.read_sql_query(sql_1, connection)

In [ ]:
data = [
    go.Bar(
        x= df['SOFT_NAME'] + ' ' + df['VERSION'],
        y= df['LEN_DESC']
    )
]
 
layout = go.Layout(
    title='Soft name and Len description',
    xaxis=dict(
        title='Soft name',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Len description',
        rangemode='nonnegative',
        autorange=True,
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)

fig = go.Figure(data=data, layout=layout)
 
soft_name_len_desc = py.iplot(fig, filename='soft-name-len-desc')

In [ ]:
soft_name_len_desc

### Вивисти кількість комп'ютерів у кожного власника

In [ ]:
sql_2 = '''
SELECT
    OWNER_OWNER_PASSPORT,
    count(*) count_computer
from computer
group by OWNER_OWNER_PASSPORT
'''

In [ ]:
df = pd.read_sql_query(sql_2, connection)

In [ ]:
data = [
    go.Pie(
        labels=df['OWNER_OWNER_PASSPORT'], 
        values=df['COUNT_COMPUTER']
    )
]

layout = go.Layout(title='Count computer in owner')

fig = go.Figure(data=data, layout=layout)

count_computer_owner = py.iplot(fig, filename='count-computer-owner')

In [ ]:
count_computer_owner

### Вивисти динаміку росту кількості вироблених деталей

In [ ]:
sql_3 = '''
SELECT
    date_create,
    COUNT(serial_number) count_hardware
FROM
    hardware
GROUP BY
    date_create
ORDER BY
    date_create
'''

In [ ]:
df = pd.read_sql_query(sql_3, connection)

In [ ]:
data = [go.Scatter(
    x=df['DATE_CREATE'],
    y=df['COUNT_HARDWARE'],
    mode='lines+markers'
)]

layout = go.Layout(
    title='Date and Count hardware',
    xaxis=dict(
        title='Date',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Count hardware',
        rangemode='nonnegative',
        autorange=True,
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)

fig = go.Figure(data=data, layout=layout)

date_count_hardware = py.iplot(fig, filename='date-count-hardware')

In [ ]:
date_count_hardware

In [ ]:
my_dboard = dashboard.Dashboard()
 
soft_name_len_desc_id = fileId_from_url(soft_name_len_desc.resource)
count_computer_owner_id = fileId_from_url(count_computer_owner.resource)
date_count_hardware_id = fileId_from_url(date_count_hardware.resource)
 
box_1 = {
    'type': 'box',
    'boxType': 'plot',
    'fileId': soft_name_len_desc_id,
    'title': 'Soft name and Len description'
}
 
box_2 = {
    'type': 'box',
    'boxType': 'plot',
    'fileId': count_computer_owner_id,
    'title': 'Count computer in owner'
}
 
box_3 = {
    'type': 'box',
    'boxType': 'plot',
    'fileId': date_count_hardware_id,
    'title': 'Date and Count hardware'
}
 
my_dboard.insert(box_1)
my_dboard.insert(box_2, 'below', 1)
my_dboard.insert(box_3, 'left', 2)

py.dashboard_ops.upload(my_dboard, 'My First Dashboard with Python')